# Ch5. back propagation

이번장에서는 해석적인 방법으로 network를 학습시켜보자

In [117]:
import numpy as np

In [86]:
def cross_entropy_error_one_hot(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

In [152]:
def _numerical_grad(f, x):
    h = 1e-4
    grad = np.zeros(x.size)

    for dim in range(x.size):
        temp = x[dim]

        x[dim] = temp - h
        f1 = f(x)

        x[dim] = temp + h
        f2 = f(x)

        grad[dim] = (f2 - f1) / (2 * h)

    return grad

def numerical_gradient_batch(f, X):
    if X.ndim == 1:
        return _numerical_grad(f, X)

    grad = np.zeros_like(X)
    for idx in range(X.shape[0]):
        grad[idx] = _numerical_grad(f, X[idx])

    return grad


def numerical_gradient_batch_v2(f, X):
    h = 1e-4
    if X.ndim == 1:
        return _numerical_grad(f, X)

    grad = np.zeros_like(X)
    for col in range(X.shape[1]):
        H = np.zeros_like(X)
        H[:, col] += h

        f1 = f(X - H)
        f2 = f(X + H)

        grad[:, col] = (f2 - f1) / (2 * h)

    return grad


In [127]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    c = np.max(a) # overflow 방지
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

### softmax-with-loss 함수 구현

In [128]:
class ReLU:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [129]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        return np.dot(self.x, self.W) + self.b
    
    def backward(self, dout):        
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        dx = np.dot(dout, self.W.T)
        
        return dx

In [130]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error_one_hot(self.y, t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [153]:
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.1):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        
        self.params['b1'] = np.zeros(hidden_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = ReLU()
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.last_layer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient_batch(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient_batch(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient_batch(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient_batch(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [112]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [136]:
iters_num=10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.01

In [157]:
net = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

point_count = 0

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
        
    grad = net.gradient(x_batch, t_batch)
    
    net.params['W1'] -= learning_rate * grad['W1']
    net.params['b1'] -= learning_rate * grad['b1']
    net.params['W2'] -= learning_rate * grad['W2']
    net.params['b2'] -= learning_rate * grad['b2']
    
    loss = net.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % 100 == 0:
        point_count += 1
        
        train_acc = net.accuracy(x_train, t_train)
        test_acc = net.accuracy(x_test, t_test)
        
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        print(f"{i}th iter... test_acc: {test_acc}, train_acc: {train_acc}, loss: {loss}")

0th iter... test_acc: 0.0559, train_acc: 0.060333333333333336, loss: 6.972066637502216
100th iter... test_acc: 0.2192, train_acc: 0.22053333333333333, loss: 8.952323715161102
200th iter... test_acc: 0.1861, train_acc: 0.18643333333333334, loss: 117.07823080074753


/var/folders/n5/n0b0brnd4cbb63xdqf_rmhvh0000gn/T/ipykernel_17146/1908689404.py:7: RuntimeWarning: divide by zero encountered in log
  return -np.sum(t * np.log(y)) / batch_size
/var/folders/n5/n0b0brnd4cbb63xdqf_rmhvh0000gn/T/ipykernel_17146/1908689404.py:7: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(t * np.log(y)) / batch_size


300th iter... test_acc: 0.0974, train_acc: 0.09751666666666667, loss: nan
400th iter... test_acc: 0.098, train_acc: 0.09871666666666666, loss: nan
500th iter... test_acc: 0.0974, train_acc: 0.09751666666666667, loss: nan
600th iter... test_acc: 0.098, train_acc: 0.09871666666666666, loss: nan


KeyboardInterrupt: 

In [156]:
x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = net.numerical_gradient(x_batch, t_batch)
grad = net.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    print(key)
    print(np.average(np.abs(grad[key])))
    print(np.average(np.abs(grad_numerical[key])))
    print(f"key: {key}, diff: {np.average(np.abs(grad_numerical[key] - grad[key]))}")
    print()

W1
0.003196811894494716
0.0033592211981492543
key: W1, diff: 0.0010888503783637704

b1
0.021327155944766592
0.023130536703197535
key: b1, diff: 0.008022261092226358

W2
0.026928771733088303
0.03654676018945757
key: W2, diff: 0.015991280841884076

b2
0.0794159059734475
0.10491142127189157
key: b2, diff: 0.04808846432309305

